In [1]:
##### CHANGE THIS TO YOUR FOLDER##################
##################################################
bigfol=r"C:\Users\Image_Processing_PC\OneDrive\OneDrive - UNSW\Image_Analysis\IMAGE ANALYSIS NO BLUR\doneandwereokay"
##################################################
#################################################






#initialise dependencies


from matplotlib import pyplot as plt
from numpy.linalg import norm
import numpy as np
from tabulate import tabulate

from scipy import sparse
from imutils.object_detection import non_max_suppression


from pathlib import Path
from skimage.color import rgb2lab, lab2rgb
from skimage.feature import match_template
from skimage.feature import peak_local_max

from os import listdir
import os
import glob
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
#import md5, sha
from PIL import Image
import rawpy
import imageio
from sklearn.neighbors import KernelDensity

from PIL import Image,ImageEnhance,ImageFilter
import PIL

from PIL.ExifTags import TAGS
import imutils

import os
#import hcluster

import re
from datetime import datetime
import pandas as pd
import time

from math import pow
import scipy.signal 
%matplotlib qt
#template matching

import cv2 as cv
import numpy as np
from scipy.stats import norm
import os
import sys
from tqdm.notebook import tqdm
from time import sleep
from skimage.filters import threshold_otsu, threshold_local
from skimage.filters import threshold_otsu, rank
from skimage.util import img_as_ubyte
from scipy.signal import find_peaks
import seaborn as sns


def process(token):
    return token['text']

#General Functions
def listdirs(folder):
## a function which lists the files in a folder and adds to a list. returns a list of folders, 
##input folder: the file path to folder
    return [
        d for d in (os.path.join(folder, d1) for d1 in os.listdir(folder))
        if os.path.isdir(d)
    ]

def labcolourfilt(image):
    #%matplotlib qt
    img_lab = cv.cvtColor(image, cv.COLOR_RGB2Lab)
    lower_l = np.array([140,0,0])
    upper_l = np.array([255,255,255])
    lab_l,lab_a,lab_b=cv.split(img_lab)
    mask = cv.inRange(img_lab, lower_l, upper_l)
    res = cv.bitwise_and(image, image, mask=mask)
    
    #plt.imshow(res)
    return res
    



def imgregfun(imagebef, imageafter):
#### A function for image registration, stolen of the internet but I can't remember where from
###Inputs: imagebef- the before image
##########imageafter- the after image
###outputs: transimaf- the translated after image
    # Open the image files.
    img1_color = imageafter  # Image to be aligned.
    img2_color = imagebef  # Reference image.

    # Convert to grayscale.
    img1 = cv.cvtColor(img1_color, cv.COLOR_BGR2GRAY)
    img2 = cv.cvtColor(img2_color, cv.COLOR_BGR2GRAY)
    height, width = img2.shape

    # Create ORB detector with 5000 features.
    orb_detector = cv.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not reqiured in this case).
    kp1, d1 = orb_detector.detectAndCompute(img1, None)
    kp2, d2 = orb_detector.detectAndCompute(img2, None)

    # Match features between the two images.
    # We create a Brute Force matcher with
    # Hamming distance as measurement mode.
    matcher = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    # Match the two sets of descriptors.
    matches = matcher.match(d1, d2)

    # Sort matches on the basis of their Hamming distance.
    matches.sort(key=lambda x: x.distance)

    # Take the top 90 % matches forward.
    matches = matches[:np.int(len(matches) * 90)]
    no_of_matches = len(matches)

    # Define empty matrices of shape no_of_matches * 2.
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
        p1[i, :] = kp1[matches[i].queryIdx].pt
        p2[i, :] = kp2[matches[i].trainIdx].pt

    # Find the homography matrix.
    homography, mask = cv.findHomography(p1, p2, cv.RANSAC)

    # Use this matrix to transform the
    # colored image wrt the reference image.
    transformed_img = cv.warpPerspective(img1_color, homography,
                                          (width, height))
    transimaf=transformed_img
    return transimaf
def imageprocessingfunction(beforefol,afterfol):
###A function for getting all the CR2 files within the before and after folders, then reading them
### and saving them on the disk as virtual images 
##########################################################################
###Inputs: beforefol: selected before folder
##########afterfol: selected after folder 
###Outputs: imaf: the images in the after folder as an array
###########imbef: the images in the before folder as an array 
###########beforeimfile: the list of before image files
###########afterimfile: the list of after image files
    # Get file list
    beforeimfile=glob.glob(beforefol+"\\"+"*.CR2")
    afterimfile=glob.glob(afterfol+"\\"+"*.CR2")
    #print(afterimfile)

    #Exifdata is just there in case you need to edit the images in a fancy way.
    imaf,labaf,imbef,labef=[],[],[],[]
    for impath in afterimfile:
        image,exifdata=   convertfilefun(impath)
        imaf.append(np.dstack((image)))
        labaf.append(exifdata)
    for impath in beforeimfile:
        image,exifdata= convertfilefun(impath)
        imbef.append(np.dstack((image)))
        labef.append(exifdata)
    return imaf,imbef,beforeimfile,afterimfile

def convertfilefun(path):
## a function which converts CR2 images to TIFF images the computer can actually read
## input: path- path to raw image
## output : an image that is readable using cv2
    with rawpy.imread(path) as raw:
        #Can fiddle with camera settings but I wouldn't reccoment it
        rgb = raw.postprocess(use_auto_wb=True,
                                #user_wb=[1.5,1.0,1.0,1.0],
                              no_auto_bright=True,
                              gamma=(2.222, 4.5),
                              chromatic_aberration=(1, 1),
                              bright=1.0,
                              dcb_enhance=True)
        #cv2.imwrite(path + '.tiff',rgb)
        # extract EXIF data to save as metadata
        metdat = Image.open(path)
        exifdata = metdat.getexif()
        image = rgb
        image = rgb.reshape(
            (1, image.shape[0], image.shape[1], image.shape[2]))
        return image, exifdata
        #plt.imsave(path + '.png',rgb)
        #g=print(path + '.png')
        #return g
        
def saveexcelfun(dimercoord,corecoord,savepath):
#Save the coordinates of everything to an excel spreadsheet. Yeah I know it's ugly. 
#input: dimercoord- the dimer coordinates
########corecoord- the core coordinates 
########savepath- the folder location where the files will be saved 
    columns=['Before Dimers xval']
    saveexcel=savepath+"."+"xlsx"
    beforedimercentres = pd.DataFrame({'Before Dimers xval':dimercoord})
    try:
        aftercorecentres = pd.DataFrame({'After core xval': corecoord})
    except:
        aftercorecentres=pd.DataFrame({'After core xval': np.array([0]), 
                                'After core yval': np.array([0])})

   
    writer = pd.ExcelWriter(saveexcel,engine='xlsxwriter')
    workbook=writer.book
    worksheet=workbook.add_worksheet('DimersPicked')
    writer.sheets['DimersPicked'] = worksheet
    worksheet2=workbook.add_worksheet('CoresPicked')
    writer.sheets['CoresPicked'] = worksheet2


    beforedimercentres.to_excel(writer,sheet_name='DimersPicked',startrow=1 , startcol=0)
    #worksheet.write_string(beforedimercentres.shape[0] + 4, 0, beforedimercentres.name)

    aftercorecentres.to_excel(writer,sheet_name='CoresPicked',startrow=1, startcol=3)
    

    writer.save()
def savetextfilefun(data,savepath,datastring):
##### A function which saves an array to a text file. Is a little buggy in that sometimes there's weird spaces. 
##### reccomend the excel save functions instead. Python struggles to re-read these text tiles

    savetextstring=savepath+datastring+".txt"
    file = open(savetextstring,"w")
    for dataentry in data:
        arr_of_strings = np.array2string(dataentry)
        file.write(arr_of_strings) 
    file.close() 
    

def load_images_from_folder(folder):
## A function which reads in images and adds them to a list of images.
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder
## note !! The images wil be read in with open cv, and will be in BGR format and will look strange unless converted
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))        
        if img is not None:
            images.append(img)
    return images
def load_images_from_foldercv(folder):
## A function which reads in images and adds them to a list of images. This function also converts them to RGB
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder

    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
       
        if img is not None:
            img= cv.cvtColor(img, cv.COLOR_BGR2RGB)
            images.append(img)
            
    return images
def load_images_from_folderhsv(folder):
## A function which reads in images and adds them to a list of images. This function also converts them to HSV
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        img= cv.cvtColor(img,  cv.COLOR_BGR2HSV)
        if img is not None:
            images.append(img)
    return images

#####################################################
###################################################
#Template matching functions

def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    # initialize the list of picked indexes	
    pick = []
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")
        
def nonmaxsuppression(xCoords,yCoords,template):
    center_coordinates=[]
    rects=[]
    rectangle_coordinates=[]
    (w, h) = template.shape[:2]
    #print(w)
    #print(h)
## stops the overcounting of variables with nonmax suppression and returns an updated list
    for (x, y) in zip(xCoords, yCoords):
    # update our list of rectangles
        rects.append((x, y, x +w, y + h))
    picked_rectangles=non_max_suppression_fast(np.array(rects),0.5)
        #I hate how opencv does rectangles, so arrange these to finds the centres
    for (startX, startY, endX, endY) in picked_rectangles:
        #the coordinates are top left hand corner, bottom right hand corner for rectangles 
        center_coordinates.append((startX+h//2,startY+h//2))
        rectangle_coordinates.append((startX, startY, endX, endY))
     
    #print("center coordinates are ",center_coordinates)
    #print("rectangle_coordinates are ", rectangle_coordinates)
    return center_coordinates,rectangle_coordinates,w,h



def blockimagecombo(image_rgb, points,r,bok):
    #print(points)
    if bok==1:
        image_blocked=image_rgb.copy()
        for pt in points:
            
            image_blocked = cv.circle(image_blocked,pt,r, (255,255,255), -1)
    else:
        mask2 = np.zeros(image_rgb.shape[:2], dtype= np.uint8)
        for pt in points: 
            #print(pt)
            mask2 = cv.circle(mask2,pt,r, (255,255,255), -1)
            
            # a rectangle is drawn on the mask, which marks where the points are 
        #invmask=255-mask2
        #This is an inbuilt cv function which clips the image around the mask. 
        #plt.imshow(cv.bitwise_and(image_rgb, image_rgb, mask=invmask))
        image_blocked = cv.bitwise_and(image_rgb, image_rgb, mask=mask2)
    return image_blocked


def performtemplatematching(image_rgb,imgpath,bok,threshold,clumpflag):
## a function which performs template matching on the images and blocks them depending on whether
## we want to keep the particles or block them out for more accuracy 
#input: image_rgb: the big image in rgb format where we are looking for matches
########imgpath: the path to the template images, as a string
#########bok: block or keep. 0 is for keeping, 1 is for blocking with a rectange
#########threshold: the threshold for the minimum values. Variable. 
    
    center_coordinates,rectangle_coordinates,w,h=imagetempmatch(imgpath,image_rgb,threshold,clumpflag)
    r=round(w/1.5)

    correctedimg=blockimagecombo(image_rgb,center_coordinates ,r,bok)
    return correctedimg,center_coordinates,r

def performsaveimage(image,path):
## saves image using pillow, which is a lot faster than matplot lib. 
    img_rgb_corr=image
    try:
        im_pil = Image.fromarray(img_rgb_corr)
        im_pil.save(path, compress_level=1)
    except:
        exception=1

def savetotrainingfol(foldername,image,points):
## saves the cropped images to a folder, for use in machine learning. uses a 16 pixel box.
#input: foldername: name of the folder where you want the images stored
#######image: an image in rgb format which you want to cut up
########points: the coordinates of the particles which you have selected. 
    boxwid=round(16/2)
    w=16
    for j,pt in enumerate(points):
        savepathfol= addstringwithtime(foldername +'\\')
        savepath= savepathfol+str(j)+"registeredimg" + "." + "png"
        #savepath=os.path.join(savepathfol, str(j)+"registeredimg" + "." + "png")
        lilimage=image[pt[1]-w:pt[1]+w, pt[0]-w:pt[0]+w]
        performsaveimage(lilimage,savepath)
        
def blockoutunwantedparticles(analpath,sat_img,path,threshold,clumpflag):
## a function which blocks out that particles which are interfering with analysis i.e. clumps and clusters
#inputs: analpath: the path to the analysis folder
######## sat_img: the before target image in rgb format
#output: correctedimgcore, the corrected image after accounting for cores and clumps
# These images are hardcoded into a folder, so the code has some dependencies. But any exmaples of the correct size will do
    #The threshold of 0.25 seems to be highly variable
    correctedimg,_,r= performtemplatematching(sat_img, path,1,threshold,clumpflag)
    #plt.imshow(correctedimg)
    #print(r)
    return correctedimg,r

         
def getthecolourscores(vector):
    #print(vector)
    splitvec=np.array_split(vector, 2,axis=1)
    #print(splitvec)
    intvals=[np.trapz(vec) for vec in splitvec]
    #print(intvals[0])
    #print(intvals[1])
    #yes=here
    
    return intvals[0],intvals[1]
def makeLabvectorspretty(l_vec,a_vec,b_vec):
    #l_vec = l_vec.ravel()
    lnoz=np.nonzero(l_vec)
    l_vecf=l_vec[lnoz]
    #a_vec = a_vec.ravel()
    #a_vec.sort()
    anoz=np.nonzero(a_vec)
    a_vecf=a_vec[anoz]
    #b_vec = b_vec.ravel()
    bnoz=np.nonzero(b_vec)
    b_vecf=b_vec[bnoz]
    return l_vecf.reshape((1,-1)),a_vecf.reshape((1,-1)),b_vecf.reshape((1,-1))
def makeLabvectorsprettyind(l_vec):
    lnoz=np.nonzero(l_vec)
    l_vecf=l_vec[lnoz]
    return l_vecf.reshape((1,-1))

def makelinspacevector(vector):
    max_vec=np.max(vector)
    min_vec=np.min(vector)
    #print(vector.shape)
    xshape,yshape=vector.shape
    
    #print(yshape)
    lin_vec = np.linspace(min_vec, max_vec,num=yshape)
    return lin_vec

def getkdefunction(vector):
    #print(vector)
    #print(vector.shape)
    
    
    kde = KernelDensity(kernel="gaussian", bandwidth=3)
    
    
    if len(vector)==0:
        log_dens=[0]
      

    else:
        kde.fit(vector.reshape((1,-1)))
        x_d=makelinspacevector(vector.reshape((1,-1)))
        log_dens = kde.score_samples(x_d.reshape((1,-1)))
        

    return log_dens


def getkdepeaksLABind(l_vec):
    vector=makeLabvectorsprettyind(l_vec)
    
    if len(l_vec)!=0:
        
        log_dens_l=getkdefunction(l_vec)
        x_d=makelinspacevector(l_vec.reshape((1,-1)))
        #density = norm(l_vec).pdf(x_d)
        #yes=here
        kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
        kde.fit(l_vec[:, None])
        logprob = kde.score_samples(x_d[:, None])
        
        #peaks_l, _ = find_peaks(logprob)
        
        intneg,intpos=getthecolourscores(vector)
        #print(log_dens_l)
        #%matplotlib qt
        #plt.figure()
        #plt.fill_between(x_d, density, alpha=0.5)
        
        #plt.plot(x_d, np.exp(logprob))
        #plt.hist(l_vec,bins=5)
        #plt.plot(density)
        #plt.hist(l_vec, bins=8)
        #plt.plot(x_d,density)
        #plt.plot(peaks_l, logprob[peaks_l], "x")
        #plt.show()
        #stop=here
        
    #if len(peaks_l)==0:
        #peaks_l=[0]
    if len(intpos)==0:
        intpos=[0.1]
        intneg=[0.1]
    
    

    
    #print(peaks_l)
    #print(peaks_b)
    #print(peaks_a)
    #plt.figure()
    #plt.plot(log_dens_a)
    #plt.plot(peaks_a, log_dens_a[peaks_a], "x")
    #plt.show()
    #yes=here
    return intneg,intpos#peaks_l    
    



def gethistogrampeaksLAB(l_vec,a_vec,b_vec):
    l_vec,a_vec,b_vec=makeLabvectorspretty(l_vec,a_vec,b_vec)
    peaks_l,peaks_a,peaks_b=getkdepeaksLAB(l_vec,a_vec,b_vec)
    
 

    return peaks_l,peaks_a,peaks_b,y_l,y_a,y_b

def processimagecrop(vector,w,pt):
    cropimage=vector[pt[1]-w:pt[1]+w, pt[0]-w:pt[0]+w]
    #print(cropimage)
    cropimr=cropimage.reshape((1,-1))
    #print(cropimr)
    return cropimr
def keepemptyindex(vector,point):
    item=vector[np.nonzero(vector)]
    pointlist=point
    return item, pointlist

def checkforempty(vector,points):
    vector2,pointlist=map(list,zip(*[keepemptyindex(item,points[index]) for index,item in enumerate(vector) if len(item[np.nonzero(item)])!=0]))
    
    return vector2,np.asarray(pointlist).reshape((-1,2))
    

def getaveragevaluesLAB(image,points,analpath,yespeaks):
    #A function which gets the colour scores for each lab colour vector by finding the vector, doing a kde desnsity graph of each value, splitting that kde density in half 
    # and then integrating each half
    ####input: image- the image you want to analyse
    ###########points- the centre of each particle you want to analyse
    ##########analpath- the place to save the analysis
    ##########yespeaks- an optional boolean so you have the option to not get the scores for each round. 
    #width of the sampling box around the particle center
    w=8
    #convert to lab colourspace
    labim = rgb2lab(image)
    #split into individual vectors
    l_vec,a_vec,b_vec = cv.split(labim)
    average_colour_aDivg=[]
    max_val_lDivg=[]
    average_lum=[]
    average_comp_a_to_b=[]
    info_l=[]
    info_a=[]
    info_b=[]
    allchannel=[l_vec,a_vec,b_vec]
    
    #Get the cropped particles around the center points
    allcropimagesl=[processimagecrop(l_vec,w,pt) for pt in points if len(l_vec)!=0]
    #print(points)
    #check that the cropped images aren't empty, delete the empty ones
    allcropimagesl2,pointlistl=checkforempty(allcropimagesl,points)
    
    
    #repeat for the a vector
    allcropimagesa=[processimagecrop(a_vec,w,pt) for pt in pointlistl]
    allcropimagesa2,pointlist_a=checkforempty(allcropimagesa,pointlistl)
   
    
    #repeat for the b vector
    allcropimagesb=[processimagecrop(b_vec,w,pt) for pt in pointlist_a]
    allcropimagesb2,pointlist_b=checkforempty(allcropimagesb,pointlist_a)
    
    #for each cropped image, perform the kde analysis and get the colour score by integration. repeat for the three vectors. 
    neg_l,pos_l=map(list,zip(*[getkdepeaksLABind(vector) for vector in allcropimagesl2 if yespeaks==1]))
  

    greenscore,redscore=map(list,zip(*[getkdepeaksLABind(vector) for vector in allcropimagesa2 if yespeaks==1]))

  
    bluescore,yellowscore=map(list,zip(*[getkdepeaksLABind(vector) for vector in allcropimagesb2 if yespeaks==1]))
   
    return neg_l,pos_l,greenscore,redscore,bluescore,yellowscore,pointlist_b



######### Extra functions

def getindividualfoldersandsuch(bigfol):
# a function which gets the folders underneath the big folder. Must be the format: bigfol>slide>sample>satellite | target> images
    topfolders=listdirs(bigfol)
    subfolders=[0]
    for nextfolders in topfolders:
        subfolders= subfolders+ listdirs(nextfolders)
    subfolders.pop(0)
    #print(subfolders)
    subfolders=np.array(subfolders).reshape(-1,1)
    #print(subfolders)
    return subfolders
def searchforsatandtargetfolders(folder):
#finds the satellite and targetfolders using regexp
    subfolders2 = folder.tolist()
    subfolders3=str(subfolders2).replace('[','').replace(']','').replace('\\\\','\\')

    listexpfolders=listdirs(subfolders3[1:len(subfolders3)-1])
    beforefol = [x for x in listexpfolders if re.search("satellite",x)]
    beforefol=beforefol[0]
    afterfol= [x for x in listexpfolders if re.search("target",x)]
    afterfol=afterfol[0]
    
    return beforefol,afterfol

def createanalysisfolder(beforefol,string):
#creates an analysis folder in the address above where the satellite and target folders are located.
    oneuppath=os.path.dirname(beforefol)
    analysisfolderpath=oneuppath+"\\"+string
    try:
        os.mkdir(analysisfolderpath)
    except:
        print("folder already exists")
    print(analysisfolderpath)
    print("processing images")
    return analysisfolderpath,oneuppath

def createimagesubfolderforsaving(pathtomatch,analysisfolderpath):
# creates a folder with the name of the image in the analysis folder 
    pathtomatch=beforeimfile[j]
    matchingsearch=re.search("IMG_.*.CR2",pathtomatch)
    savefilespath=analysisfolderpath+"\\"+matchingsearch.group()+"\\"
    try: 
        os.mkdir(savefilespath)
    except:
        print("folder already exists")
    return savefilespath
def addstringwithtime(savefilespath):
# adds the current date and time so there's no saving over the top of different analysis
    now = datetime.now()
    dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
    savepath=savefilespath+dt_string
    return savepath
def makeanalysisfolder(savefilespath,string):
# makes a directory in the folder which matches the image name which says 'analysis'
    analpath=os.path.join(savefilespath,string)
                        #saves the registered image
    try: 
        os.mkdir(analpath)
    except:
        print("folder already exists")
    
    return analpath

################ THRESHOLDING

def getcentroidsandcenters(image):
    ### a function which gets the centres of particles. It's not perfect but it's okay
    
    h=8
    src=image.copy()
    #turns the image black and white
    bw = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    #_, bw = cv.threshold(bw, 40, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    #Finds the contours of the black and white image
    contours, hierarchy = cv.findContours(bw, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    list_of_centres=[]
    boundRect = [None]*len(contours)
    contours_poly = [None]*len(contours)
    #filters for repeats of the same point by drawing a rectangle around it and seeing if it overlaps using non_max_suppression
    for i, c in enumerate(contours):
        contours_poly[i] = cv.approxPolyDP(c, 3, True)
        center_circle, _ = cv.minEnclosingCircle(contours_poly[i])
        boundRect[i] = cv.boundingRect(contours_poly[i])
    picked_rectangles=non_max_suppression_fast(np.array(boundRect),0.5) 
   
    
        #I hate how opencv does rectangles, so arrange these to finds the centres
    for (startX, startY, endX, endY) in picked_rectangles:
        #the coordinates are top left hand corner, bottom right hand corner for rectangles 
        list_of_centres.append((startX+h//2,startY+h//2))
        #rectangle_coordinates.append((startX, startY, endX, endY))
    #list_of_centres.append((np.int(np.round(center_circle[0])),np.int(np.round(center_circle[1]))))
    for centers in list_of_centres:
        cv.circle(src, ((np.int(np.round(centers[0])),np.int(np.round(centers[1])))), 20, (255, 255, 0), 1)

        
    return list_of_centres

def returnlistoftargeteddimers(scorelist,greenscore,redscore):
    ### just a function that returns the filtered values for a list of attributes
    returnedlist=np.array(scorelist)[np.logical_and(np.array(greenscore)>100,np.array(redscore)>200)]
    
    return returnedlist
def returnlistoftargetedcores(scorelist,greenscore,redscore):
    ### just a function that returns the filtered values for a list of attributes
    returnedlist=np.array(scorelist)[np.logical_and(np.array(greenscore)<-50,np.array(redscore)<-100)]
    
    return returnedlist

def drawcirclesandsave(image,points,analpath,string):
    #####A function which draws circles on an image given some points. Truly thrilling lol
    ##inputs: image- the image you're drawing circles on
    #########points- the centers of the circles you want to draw
    ########analpath- the analysis path where you want to save the image
    ########string- a description of the image
    circleim=image.copy()
    for pt in points:
        cv.circle(circleim, (pt[0],pt[1]), 8, (255, 255, 0), 4)
    savepath=analpath+'\\'+string
    performsaveimage(circleim,savepath)
    
def performthresholdingLABfortarget(beforeim,afterim,analpath):
    ### performs the thresholding using LAB colour vectors. Integrates these vectors for each particle to come up with colourscores for each particle, then filtered.
    ##inputs: beforeim- dimer image
    #########afterim-target image
    ##### used for target data, change if you want red to green instead of green to red
    ##outputs: list_of_data- colourscores for each target particle selected 
    ##########dimer_points2- the centres of each particle for each target particle selected
    ##########listofdatabefore-the colourscores for each dimer particle corresponding to the selected target
    ##########core_points2- confusing, sorry, will fix eventually. The centers of each selected dimer particle
    #gets the list of centers of all particles using the centroid function on the before images
    list_of_centres=getcentroidsandcenters(beforeim)
    #For each center found, get the corresponding colour and luminance values
    neg_l,pos_l,greenscore,redscore,bluescore,yellowscore,pointlist_b=getaveragevaluesLAB(beforeim,list_of_centres,analpath,1)
    #filter the selected points using green and red score parameters
    filtercore=np.asarray(np.logical_and(np.array(greenscore)>100,np.array(redscore)>200))
    #it goes funny if I try normally filtering it so I have to do this yuck version
    core_points=np.array(pointlist_b)*filtercore
    core_points2=(core_points[core_points!=0]).reshape((-1,2))
    #Draw the circles of the selected particles on the image and save it in the analysis folder
    drawcirclesandsave(beforeim,core_points2,analpath,"dimers_selected.png")
    print("done the dimer points")
    
    #Do the same process for the target, using the dimer points from before and seeing if any dimer particles have gone green
    neg_l_d,pos_l_d,greenscore_d,redscore_d,bluescore_d,yellowscore_d,pointlist_d=getaveragevaluesLAB(afterim,core_points2,analpath,1)
    #print("done the dimer points")
    filterdimer=np.asarray(np.logical_and(np.asarray(greenscore_d).reshape((-1,1))<-50,np.asarray(redscore_d).reshape((-1,1))<-100))
    dimer_points=np.array(pointlist_d)*filterdimer
    dimer_points2=(dimer_points[dimer_points!=0]).reshape((-1,2))
    drawcirclesandsave(afterim,dimer_points2,analpath,"target_selected.png")
    #targeted_points=np.array(core_points).reshape((-1,2))[np.logical_and(np.asarray(greenscore).reshape((-1,1))>100,np.asarray(redscore).reshape((-1,1))>100)]    
    neg_l_c,pos_l_c,greenscore_c,redscore_c,bluescore_c,yellowscore_c,pointlist_c=getaveragevaluesLAB(beforeim,dimer_points2,analpath,1)
    arrayofdata=[neg_l_d,pos_l_d,greenscore_d,redscore_d,bluescore_d,yellowscore_d]
    list_of_data=[returnlistoftargetedcores(scorelist,greenscore_d,redscore_d) for scorelist in arrayofdata]
    listofdatabefore=[neg_l_c,pos_l_c,greenscore_c,redscore_c,bluescore_c,yellowscore_c]

    return list_of_data,core_points2,dimer_points2,listofdatabefore
    
    
def performthresholdingLABforruler(beforeim,afterim,analpath):
    ### performs the thresholding using LAB colour vectors. Integrates these vectors for each particle to come up with colourscores for each particle, then filtered.
    ##inputs: beforeim- core image
    #########afterim-dimer image
    ##### used for ruler data, change if you want green to red instead of red to green
    ##outputs: list_of_data- colourscores for each dimer particle selected 
    ##########dimer_points2- the centres of each particle for each dimer particle selected
    ##########listofdatabefore-the colourscores for each core particle corresponding to the selected dimer
    
    list_of_centres=getcentroidsandcenters(beforeim)
    
    neg_l,pos_l,greenscore,redscore,bluescore,yellowscore,pointlist_b=getaveragevaluesLAB(beforeim,list_of_centres,analpath,1)
    filtercore=np.asarray(np.logical_and(np.array(greenscore)<-50,np.array(redscore)<-100))
    core_points=np.array(pointlist_b)*filtercore
    
    core_points2=(core_points[core_points!=0]).reshape((-1,2))
    drawcirclesandsave(beforeim,core_points2,analpath,"cores_selected.png")
    
    
    print("done the core points")
    neg_l_d,pos_l_d,greenscore_d,redscore_d,bluescore_d,yellowscore_d,pointlist_d=getaveragevaluesLAB(afterim,core_points2,analpath,1)
    #print("done the dimer points")
    filterdimer=np.asarray(np.logical_and(np.asarray(greenscore_d).reshape((-1,1))>100,np.asarray(redscore_d).reshape((-1,1))>200))
    dimer_points=np.array(pointlist_d)*filterdimer
    dimer_points2=(dimer_points[dimer_points!=0]).reshape((-1,2))
    drawcirclesandsave(afterim,dimer_points2,analpath,"dimers_selected.png")
    #targeted_points=np.array(core_points).reshape((-1,2))[np.logical_and(np.asarray(greenscore).reshape((-1,1))>100,np.asarray(redscore).reshape((-1,1))>100)]
    
    neg_l_c,pos_l_c,greenscore_c,redscore_c,bluescore_c,yellowscore_c,pointlist_c=getaveragevaluesLAB(beforeim,dimer_points2,analpath,1)
    
    arrayofdata=[neg_l_d,pos_l_d,greenscore_d,redscore_d,bluescore_d,yellowscore_d]
    list_of_data=[returnlistoftargeted(scorelist,greenscore_d,redscore_d) for scorelist in arrayofdata]
    
    listofdatabefore=[neg_l_c,pos_l_c,greenscore_c,redscore_c,bluescore_c,yellowscore_c]
    
    
    
    return list_of_data,dimer_points2,listofdatabefore
subfolders=getindividualfoldersandsuch(bigfol)

for folder in subfolders:
    #automatically find the before and after folder 
    beforefol,afterfol= searchforsatandtargetfolders(folder)
    
    #Create the relevant folder for saving,as well as the path one folder up 
    #analysisfolderpath_temp,oneuppath_temp=createanalysisfolder(beforefol,"Analysisfolder_Template")
    analysisfolderpath_thresh,oneuppath_thresh=createanalysisfolder(beforefol,"Analysisfolder_thresholding")
    
    
    #loadsall the images in 

    imaf,imbef,beforeimfile,afterimfile=imageprocessingfunction(beforefol,afterfol)
    print("processed and converted images")
    
    
    #response=[0]*len(beforeimfile)
    #dimercount=[0]*len(beforeimfile)
    #targetcount=[0]*len(beforeimfile)
    response_thresh=[0]*len(beforeimfile)
    dimercount_thresh=[0]*len(beforeimfile)
    targetcount_thresh=[0]*len(beforeimfile)
    dimer_greenscore=[0]*len(beforeimfile)
    dimer_redscore=[0]*len(beforeimfile)
    dimer_yellowscore=[0]*len(beforeimfile)
    dimer_bluescore=[0]*len(beforeimfile)
    dimer_lscore=[0]*len(beforeimfile)
    target_greenscore=[0]*len(beforeimfile)
    target_redscore=[0]*len(beforeimfile)
    target_yellowscore=[0]*len(beforeimfile)
    target_bluescore=[0]*len(beforeimfile)
    target_lscore=[0]*len(beforeimfile)
    #t = tqdm(total=len(imbef))
    for j,image in enumerate(imbef):
        

        pathtomatch=beforeimfile[j]
        #savefilespath_temp= createimagesubfolderforsaving(pathtomatch,analysisfolderpath_temp)
        savefilespath_thresh= createimagesubfolderforsaving(pathtomatch,analysisfolderpath_thresh)
        
        
        #add on the currentdate and time 
        #savepath_temp=addstringwithtime(savefilespath_temp)
        savepath_thresh=addstringwithtime(savefilespath_thresh)
        
        
        print("calculating image "+str(j+1)+" of "+ str(len(imbef)))
        
        #make an analysis folder which says analysis in the image folder just created 
        #analpath_temp=makeanalysisfolder(savefilespath_temp,"Analysisimages_Template")
        analpath_thresh=makeanalysisfolder(savefilespath_thresh,"Analysisimages_thresholding")
        
        #spot matches the before and after images 
        transimaf=imgregfun(image, imaf[j])
        #saves the redistered image
        saveregisteredimage=os.path.join(analpath_thresh, "registeredimg" + "." + "png")
        performsaveimage(transimaf,saveregisteredimage)
       
        #uncomment if you want a crop box    
        #height, width = image.shape[:2]
        #boxwid=round(1500/2)
        #centreimagebefore=image[round(height/2)-boxwid:round(height/2)+boxwid,round(width/2)-boxwid:round(width/2)+boxwid]
        #centreimageafter=transimaf[round(height/2)-boxwid:round(height/2)+boxwid,round(width/2)-boxwid:round(width/2)+boxwid]
        centreimagebefore=labcolourfilt(image)
        
        print("Made the clipping Mask")
        #plt.imshow(centreimagebefore)
        centreimageafter=labcolourfilt(transimaf)
        
        #with tqdm(total=len(imbef), file=sys.stdout) as pbar:

            #pbar.write('processed: %d' % (j+1))
            #pbar.update(0.75)
            #sleep(0.1)
        list_of_data,dimer_points,target_points,listofdatabefore=performthresholdingLABfortarget(centreimagebefore,centreimageafter,analpath_thresh)
        #print(np.array(list_of_data).shape)
        neg_l_d,pos_l_d,greenscore_d,redscore_d,bluescore_d,yellowscore_d=np.vsplit(np.array(listofdatabefore), 6)
        #print(neg_l_d.shape)
        neg_l,pos_l,greenscore,redscore,bluescore,yellowscore=np.vsplit(np.array(list_of_data), 6)

        #print(neg_l.shape)
        #print(neg_l.reshape((-1,1)))

        neg_l=neg_l.reshape((-1,1))
        pos_l=pos_l.reshape((-1,1))
        greenscore=greenscore.reshape((-1,1))
        redscore=redscore.reshape((-1,1))
        bluescore=bluescore.reshape((-1,1))
        yellowscore=yellowscore.reshape((-1,1))

        neg_l_d=neg_l.reshape((-1,1))
        pos_l_d=pos_l.reshape((-1,1))
        greenscore_d=greenscore_d.reshape((-1,1))
        redscore_d=redscore_d.reshape((-1,1))
        bluescore_d=bluescore_d.reshape((-1,1))
        yellowscore_d=yellowscore_d.reshape((-1,1))
        
        
        
        
        #saves the to an excel spreadsheet and saves the results to a list to get the overall results at the end 
        #if I feel bothered I can add something in that uses the already analysed images but meh 

        
        dimercount_thresh[j]=len(dimer_points)
        targetcount_thresh[j]=len(target_points)
        saveexpath=analpath_thresh+'\\'+"responseforallimages.xlsx"

        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')
        dimerresp=pd.DataFrame({'dimer_greenscore':greenscore_d,'dimer_yellowscore':yellowscore_d,'dimer_redscore':redscore_d,'dimer_bluescore':bluescore_d,'dimer_luminosity':neg_l_d})
        targresp=pd.DataFrame({'target_greenscore':greenscor,'target_yellowscore':yellowscore,'target_redscore':redscore,'target_bluescore':bluescore,'target_luminosity':neg_l})
        # Write each dataframe to a different worksheet.
        dimerresp.to_excel(writer, sheet_name='DimerData')
        targetresp.to_excel(writer, sheet_name='Sheet2')
        

        # Close the Pandas Excel writer and output the Excel file.
        writer.save()
        try: 
            response_thresh[j]=(len(target_points)/len(dimer_points))*100
            
        except:
            response_thresh[j]=0

  
 
    print("Finished analysis woohoo")
    #t.close()
    
    searchfolder_thresh=oneuppath_thresh
    #insearchfolder=os.listdir(searchfolder)
    savedir_thresh= searchfolder_thresh+'\\'+"Analysisfolder_thresholding"
    imagefilefolders=os.listdir(savedir_thresh)
    #Write excel worsheet in the analysis folder
    #print(dimercount_thresh)
    
    responseforexcel = pd.DataFrame({'dimers_picked':np.array(dimercount_thresh),'target_picked':np.array(targetcount_thresh),'response': np.array(response_thresh)})
    
    responsepath_thresh=savedir_thresh+'\\'+"responseforallimages.xlsx"

    responseforexcel.to_excel(responsepath_thresh) 


folder already exists
C:\Users\Image_Processing_PC\OneDrive\OneDrive - UNSW\Image_Analysis\IMAGE ANALYSIS NO BLUR\doneandwereokay\slide1-11120-cal2slides\1-10fM\Analysisfolder_thresholding
processing images
processed and converted images
folder already exists
calculating image 1 of 6
folder already exists
Made the clipping Mask


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:329: RuntimeWarning: invalid value encountered in true_divide


done the dimer points
folder already exists
calculating image 2 of 6
folder already exists
Made the clipping Mask


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:329: RuntimeWarning: invalid value encountered in true_divide


done the dimer points
folder already exists
calculating image 3 of 6
folder already exists
Made the clipping Mask


...

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:329: RuntimeWarning: invalid value encountered in true_divide


done the dimer points
Finished analysis woohoo
folder already exists
C:\Users\Image_Processing_PC\OneDrive\OneDrive - UNSW\Image_Analysis\IMAGE ANALYSIS NO BLUR\doneandwereokay\slide1-11120-cal2slides\4-10fM\Analysisfolder_thresholding
processing images
processed and converted images
folder already exists
calculating image 1 of 4
folder already exists
Made the clipping Mask
done the dimer points
folder already exists
calculating image 2 of 4
folder already exists
Made the clipping Mask


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:329: RuntimeWarning: invalid value encountered in true_divide


done the dimer points


ValueError: not enough values to unpack (expected 2, got 0)

[1997, 571, 890, 981, 579, 1936]
